# About Dataset
### Context
A small subset of dataset of product reviews from Amazon Kindle Store category.

### Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.

### Columns
<ul>
  <li>asin - ID of the product, like B000FA64PK</li>
  <li>helpful - helpfulness rating of the review - example: 2/3.</li>
  <li>overall - rating of the product.</li>
  <li>reviewText - text of the review (heading).</li>
  <li>reviewTime - time of the review (raw).</li>
  <li>reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN</li>
  <li>reviewerName - name of the reviewer.</li>
  <li>summary - summary of the review (description).</li>
  <li>unixReviewTime - unix timestamp.</li>
</ul>




### Which file to use?
The dataset originally contained a json file of the reviews, but some people had issues opening it and getting it to work so I've added a csv file which contains same data. You can use whichever one is easier to work with.

### Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

### Inspiration
<ul>
  <li>Sentiment analysis on reviews.</li>
  <li>Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.</li>
  <li>Fake reviews/ outliers.</li>
  <li>best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).</li>
  <li>Any other interesting analysis.</li>
  
</ul>






## Best Practises
1. Preprocessing and  Cleaning
2. Train Test Split
3. BOW, TFIDF, Word1vec
4. Train ML Algorithm

In [45]:
# Load The Dataset
import pandas as pd
df = pd.read_csv('DataSet/Kindle Reviews/all_kindle_review .csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [46]:
df[['reviewText','rating']]

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [47]:
df.shape

(12000, 11)

In [48]:
## Misisng Values
df.isnull().sum()

Unnamed: 0.1       0
Unnamed: 0         0
asin               0
helpful            0
rating             0
reviewText         0
reviewTime         0
reviewerID         0
reviewerName      38
summary            2
unixReviewTime     0
dtype: int64

In [49]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [50]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Preprocessing and Cleaning

In [51]:
## Positive review is 1 and Negetive review is 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [52]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [53]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [54]:
## 1. Lower All The Cases
df['reviewText'] = df['reviewText'].str.lower()

In [55]:
df[['reviewText','rating']]

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1
...,...,...
11995,valentine cupid is a vampire- jena and ian ano...,1
11996,i have read all seven books in this series. ap...,1
11997,this book really just wasn't my cuppa. the si...,1
11998,"tried to use it to charge my kindle, it didn't...",0


In [56]:
!pip install beautifulsoup4 lxml
!pip install lxml
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sahic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
## Removing Special Character
df['reviewText'] = df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+','',x))

## Removing the Stopwords
df['reviewText'] = df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))

df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://[\w.-]+(?:\.[\w.-]+)*(?:[\w.,@?^=%&:/~+#-]*)?', '', x))

## Removing HTML tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

## Removing any Additional Spaces
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))





In [58]:
df[['reviewText','rating']]

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1
...,...,...
11995,valentine cupid vampire jena ian another vampi...,1
11996,read seven books series apocalypticadventure o...,1
11997,book really wasnt cuppa situation man capturin...,1
11998,tried use charge kindle didnt even register ch...,0


In [60]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [61]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [62]:
df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))

In [63]:
df[['reviewText','rating']]

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1
...,...,...
11995,valentine cupid vampire jena ian another vampi...,1
11996,read seven book series apocalypticadventure on...,1
11997,book really wasnt cuppa situation man capturin...,1
11998,tried use charge kindle didnt even register ch...,0


## Train Test Split

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.20)

In [69]:
## Bag OF Words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [70]:
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [71]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [73]:
## Machine Learning  Algorithm
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf,y_train)


In [74]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [75]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

## For BOW

In [79]:
confusion_matrix(y_test,y_pred_bow)

array([[525, 269],
       [717, 889]], dtype=int64)

In [77]:
print("BOW Accuracy : ",accuracy_score(y_test,y_pred_bow))

BOW Accuracy :  0.5891666666666666


## For TF-IDF

In [80]:
confusion_matrix(y_test,y_pred_tfidf)

array([[513, 281],
       [697, 909]], dtype=int64)

In [78]:
print("TF-IDF Accuracy : ",accuracy_score(y_test,y_pred_tfidf))

TF-IDF Accuracy :  0.5925
